In [1]:
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
import pandas as pd

In [2]:
bcf = pd.read_excel("BCF.xlsx")
ps_bart = pd.read_excel("ps_BART.xlsx")

In [3]:
bcf.columns

Index(['empirical_ATE_RMSE', 'empirical_ATE_MAE', 'empirical_ATE_MAPE',
       'empirical_ATE_len', 'empirical_ATE_cover', 'theoretical_CATE_RMSE',
       'theoretical_CATE_MAE', 'theoretical_CATE_MAPE', 'theoretical_CATE_len',
       'theoretical_CATE_cover'],
      dtype='object')

In [4]:
# Calculate the mean and standard deviation for each column
mean_values = bcf.mean()
std_values = bcf.std()

# Print the results
print("Mean of each column:")
print(mean_values)
print("\nStandard deviation of each column:")
print(std_values)

Mean of each column:
empirical_ATE_RMSE        103.099016
empirical_ATE_MAE          51.027826
empirical_ATE_MAPE          5.553600
empirical_ATE_len          69.375239
empirical_ATE_cover         0.510000
theoretical_CATE_RMSE     345.630202
theoretical_CATE_MAE      216.295430
theoretical_CATE_MAPE       7.277011
theoretical_CATE_len      127.600044
theoretical_CATE_cover      0.402114
dtype: float64

Standard deviation of each column:
empirical_ATE_RMSE         17.128290
empirical_ATE_MAE           7.031763
empirical_ATE_MAPE          3.505429
empirical_ATE_len          17.397900
empirical_ATE_cover         0.181358
theoretical_CATE_RMSE     105.136017
theoretical_CATE_MAE       65.217650
theoretical_CATE_MAPE       3.547586
theoretical_CATE_len       18.871732
theoretical_CATE_cover      0.037672
dtype: float64


In [5]:
bcf.head()

,empirical_ATE_RMSE,empirical_ATE_MAE,empirical_ATE_MAPE,empirical_ATE_len,empirical_ATE_cover,theoretical_CATE_RMSE,theoretical_CATE_MAE,theoretical_CATE_MAPE,theoretical_CATE_len,theoretical_CATE_cover
0,96.010901,51.048437,4.941559,66.582021,0.388,288.151897,182.068926,6.629772,139.223760,0.437496
1,66.214528,48.333129,5.904967,69.055880,0.338,199.201953,125.154120,6.667884,92.311390,0.400016
2,88.384317,60.854685,8.810337,116.980024,0.468,258.281777,163.018209,20.322190,137.817051,0.459744
3,84.999113,46.562314,3.585140,71.018727,0.468,257.391214,161.192951,7.187570,126.329141,0.440784
4,86.204786,72.492547,13.620545,78.665890,0.202,225.394013,140.884645,6.903871,118.938082,0.441864


In [6]:
# Calculate RSE and AE for 'empirical_ATE_cover' and 'theoretical_CATE_cover'
bcf_SE_empirical_ATE_cover = (((0.95 - bcf['empirical_ATE_cover'])**2))
bcf_AE_empirical_ATE_cover = (np.abs(0.95 - bcf['empirical_ATE_cover']))

bcf_SE_theoretical_CATE_cover = (((0.95 - bcf['theoretical_CATE_cover'])**2))
bcf_AE_theoretical_CATE_cover = (np.abs(0.95 - bcf['theoretical_CATE_cover']))

In [7]:
ps_bart.columns

Index(['empirical_ATE_RMSE', 'empirical_ATE_MAE', 'empirical_ATE_MAPE',
       'empirical_ATE_len', 'empirical_ATE_cover', 'theoretical_CATE_RMSE',
       'theoretical_CATE_MAE', 'theoretical_CATE_MAPE', 'theoretical_CATE_len',
       'theoretical_CATE_cover'],
      dtype='object')

In [8]:
# Calculate the mean and standard deviation for each column
mean_values = ps_bart.mean()
std_values = ps_bart.std()

# Print the results
print("Mean of each column:")
print(mean_values)
print("\nStandard deviation of each column:")
print(std_values)

Mean of each column:
empirical_ATE_RMSE         5.302814
empirical_ATE_MAE          2.784038
empirical_ATE_MAPE         0.316423
empirical_ATE_len         13.786408
empirical_ATE_cover        0.938460
theoretical_CATE_RMSE     21.211280
theoretical_CATE_MAE      11.940961
theoretical_CATE_MAPE      0.513379
theoretical_CATE_len      57.050255
theoretical_CATE_cover     0.887205
dtype: float64

Standard deviation of each column:
empirical_ATE_RMSE         2.610444
empirical_ATE_MAE          0.611368
empirical_ATE_MAPE         0.188006
empirical_ATE_len          1.826212
empirical_ATE_cover        0.029181
theoretical_CATE_RMSE     15.203708
theoretical_CATE_MAE       7.029541
theoretical_CATE_MAPE      0.294695
theoretical_CATE_len      29.583738
theoretical_CATE_cover     0.029417
dtype: float64


In [9]:
# Calculate RSE and AE for 'empirical_ATE_cover' and 'theoretical_CATE_cover'
ps_bart_SE_empirical_ATE_cover = (((0.95 - ps_bart['empirical_ATE_cover'])**2))
ps_bart_AE_empirical_ATE_cover = (np.abs(0.95 - ps_bart['empirical_ATE_cover']))

ps_bart_SE_theoretical_CATE_cover = (((0.95 - ps_bart['theoretical_CATE_cover'])**2))
ps_bart_AE_theoretical_CATE_cover = (np.abs(0.95 - ps_bart['theoretical_CATE_cover']))

In [10]:
# Function to check normality using Shapiro-Wilk test
def check_normality(data):
    stat, p = stats.shapiro(data)
    return p > 0.05  # If p > 0.05, data is normally distributed

# Function to check homoscedasticity using Levene's test
def check_homoscedasticity(data1, data2):
    stat, p = stats.levene(data1, data2)
    return p > 0.05  # If p > 0.05, variances are equal



# ATE RMSE

In [11]:
i=0
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For empirical_ATE_RMSE
Mann-Whitney U Test:
Statistic: 0.0, p-value: 2.562143669163401e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 149.25373134328356, p-value: 2.523939423990366e-34
Levene's Test:
Statistic: 151.46059934530462, p-value: 3.200696179266804e-26
Brown-Forsythe Test:
Statistic: 151.46059934530462, p-value: 3.200696179266804e-26


# ATE MAE

In [12]:
i=1
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For empirical_ATE_MAE
Mann-Whitney U Test:
Statistic: 0.0, p-value: 2.562143669163401e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 149.25373134328356, p-value: 2.523939423990366e-34
Levene's Test:
Statistic: 111.41649996874189, p-value: 5.985890974212321e-21
Brown-Forsythe Test:
Statistic: 111.41649996874189, p-value: 5.985890974212321e-21


# ATE MAPE

In [13]:
i=2
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For empirical_ATE_MAPE
Mann-Whitney U Test:
Statistic: 8.0, p-value: 3.2576821808361544e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 148.77650149253736, p-value: 3.2091828696729075e-34
Levene's Test:
Statistic: 94.98193341763063, p-value: 1.3991170820922962e-18
Brown-Forsythe Test:
Statistic: 94.98193341763063, p-value: 1.3991170820922962e-18


# CATE RMSE

In [14]:
i=5
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For theoretical_CATE_RMSE
Mann-Whitney U Test:
Statistic: 0.0, p-value: 2.562143669163401e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 149.25373134328356, p-value: 2.523939423990366e-34
Levene's Test:
Statistic: 89.22726083779813, p-value: 1.0179908720505012e-17
Brown-Forsythe Test:
Statistic: 89.22726083779813, p-value: 1.0179908720505012e-17


# CATE MAE

In [15]:
i=6
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For theoretical_CATE_MAE
Mann-Whitney U Test:
Statistic: 0.0, p-value: 2.562143669163401e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 149.25373134328356, p-value: 2.523939423990366e-34
Levene's Test:
Statistic: 92.78730655001553, p-value: 2.968040985687189e-18
Brown-Forsythe Test:
Statistic: 92.78730655001553, p-value: 2.968040985687189e-18


# CATE MAPE

In [16]:
i=7
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For theoretical_CATE_MAPE
Mann-Whitney U Test:
Statistic: 0.0, p-value: 2.562143669163401e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 149.25373134328356, p-value: 2.523939423990366e-34
Levene's Test:
Statistic: 33.98585724090284, p-value: 2.225652383832073e-08
Brown-Forsythe Test:
Statistic: 33.98585724090284, p-value: 2.225652383832073e-08


# ATE len

In [17]:
i=3
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For empirical_ATE_len
Mann-Whitney U Test:
Statistic: 0.0, p-value: 2.562143669163401e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 149.25373134328356, p-value: 2.523939423990366e-34
Levene's Test:
Statistic: 46.76526378445827, p-value: 9.712732949425427e-11
Brown-Forsythe Test:
Statistic: 46.76526378445827, p-value: 9.712732949425427e-11


# CATE len

In [18]:
i=8
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For theoretical_CATE_len
Mann-Whitney U Test:
Statistic: 363.0, p-value: 9.451998329911333e-30
Wilcoxon Signed-Rank Test:
Statistic: 24.0, p-value: 8.023842397878198e-18
Kruskal-Wallis H Test:
Statistic: 128.3687701492537, p-value: 9.321083916772004e-30
Levene's Test:
Statistic: 1.6735358618754899, p-value: 0.19729268161256117
Brown-Forsythe Test:
Statistic: 1.6735358618754899, p-value: 0.19729268161256117


# ATE Cover SE

In [19]:
group1 = np.array(ps_bart_SE_empirical_ATE_cover)
group2 = np.array(bcf_SE_empirical_ATE_cover)
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For ATE Cover SE")
    print(f"ATE Cover RMSE for ps-BART: {np.sqrt(np.mean(group1))} and BCF: {np.sqrt(np.mean(group2))}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For ATE Cover SE")
    print(f"ATE Cover RMSE for ps-BART: {np.sqrt(np.mean(group1))} and BCF: {np.sqrt(np.mean(group2))}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For ATE Cover SE
ATE Cover RMSE for ps-BART: 0.031244199461660054 and BCF: 0.47556467488660253
Mann-Whitney U Test:
Statistic: 9.0, p-value: 3.269262957123811e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 148.76945561658698, p-value: 3.2205838300386523e-34
Levene's Test:
Statistic: 339.12200877488783, p-value: 8.796112083773729e-45
Brown-Forsythe Test:
Statistic: 339.12200877488783, p-value: 8.796112083773729e-45


# ATE Cover AE

In [20]:
group1 = np.array(ps_bart_AE_empirical_ATE_cover)
group2 = np.array(bcf_AE_empirical_ATE_cover)
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For ATE Cover AE")
    print(f"ATE Cover MAE for ps-BART: {(np.mean(group1))} and BCF: {(np.mean(group2))}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For ATE Cover AE")
    print(f"ATE Cover MAE for ps-BART: {(np.mean(group1))} and BCF: {(np.mean(group2))}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For ATE Cover AE
ATE Cover MAE for ps-BART: 0.022619999999999977 and BCF: 0.44
Mann-Whitney U Test:
Statistic: 9.0, p-value: 3.269262957123811e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.895021439772567e-18
Kruskal-Wallis H Test:
Statistic: 148.76945561658698, p-value: 3.2205838300386523e-34
Levene's Test:
Statistic: 209.09230404532968, p-value: 8.053066474465405e-33
Brown-Forsythe Test:
Statistic: 209.09230404532968, p-value: 8.053066474465405e-33


# CATE Cover SE

In [21]:
group1 = np.array(ps_bart_SE_theoretical_CATE_cover)
group2 = np.array(bcf_SE_theoretical_CATE_cover)
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For CATE Cover SE")
    print(f"CATE Cover RMSE for ps-BART: {np.sqrt(np.mean(group1))} and BCF: {np.sqrt(np.mean(group2))}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For CATE Cover SE")
    print(f"CATE Cover RMSE for ps-BART: {np.sqrt(np.mean(group1))} and BCF: {np.sqrt(np.mean(group2))}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For CATE Cover SE
CATE Cover RMSE for ps-BART: 0.06928138796762082 and BCF: 0.5491669011975139
Mann-Whitney U Test:
Statistic: 0.0, p-value: 2.5617107242843606e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 149.25406717333047, p-value: 2.5235128500228185e-34
Levene's Test:
Statistic: 115.41345164003317, p-value: 1.6616841847499023e-21
Brown-Forsythe Test:
Statistic: 115.41345164003317, p-value: 1.6616841847499023e-21


# CATE Cover AE

In [22]:
group1 = np.array(ps_bart_AE_theoretical_CATE_cover)
group2 = np.array(bcf_AE_theoretical_CATE_cover)
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For CATE Cover AE")
    print(f"CATE Cover MAE for ps-BART: {(np.mean(group1))} and BCF: {(np.mean(group2))}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For CATE Cover AE")
    print(f"CATE Cover MAE for ps-BART: {(np.mean(group1))} and BCF: {(np.mean(group2))}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For CATE Cover AE
CATE Cover MAE for ps-BART: 0.06279471999999996 and BCF: 0.54788624
Mann-Whitney U Test:
Statistic: 0.0, p-value: 2.5617107242843606e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 149.25406717333047, p-value: 2.5235128500228185e-34
Levene's Test:
Statistic: 6.259935152164844, p-value: 0.013159703067056498
Brown-Forsythe Test:
Statistic: 6.259935152164844, p-value: 0.013159703067056498
